In [70]:
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.0,
    max_tokens=300,
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY")
)

# 프롬프트 템플릿 정의
template = """질문: {question}

주의사항: 아래의 답변은 SQL 쿼리문으로만 작성됩니다. 다른 설명이나 답변 없이 SQL 쿼리문만 생성해 주세요.

SQL 답변: """
prompt = PromptTemplate.from_template(template)

# LLM Chain 객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [71]:

# 예시 질문 실행
question = """
Table Schema
ch_acq_TB (
    event_date DATE, -- 이벤트 발생 날짜 (예: 20231016)
    source_medium STRING, -- 유저 유입 경로 (UTM 기반) (예: youtube/video)
    campaign STRING, -- 상세 이벤트명 (UTM 기반) (예: EastAfrica_Nov2022)
    content STRING, -- 이벤트 내용 (UTM 기반) (예: girlseducation)
    term STRING, -- 타겟팅 검색어 (예: 유저가 검색했으면 하는 검색어)
    page_location STRING, -- 페이지 URL
    user_id STRING, -- 정기 후원자 ID 또는 일시 후원자 ID (예: 2023102263)
    session STRING, -- GA 세션 ID (예: user_pseudo_id + ga_session_id)
    user STRING, -- 고유한 유저 ID (예: 110417617.169768)
    page_bounce INTEGER, -- 페이지 이탈 횟수 (예: 0, 1, 2)
    begin_checkout INTEGER, -- 후원 신청 페이지 도달 시 발생하는 이벤트 (예: null, 1)
    regular_purchase STRING, -- 정기 후원 여부
    once_purchase STRING, -- 일시 후원 여부
    regular_value INTEGER, -- 정기 후원 금액
    once_value INTEGER -- 일시 후원 금액
);

Generate an appropriate SQL query based on the question.
Don’t hesitate to create subqueries if necessary.
You can Answer Only SQL query.

Question: 이번달 2일 youtube/video 후원수를 알려줘.
    
"""

In [72]:
def parse_sql_response(response: str) -> str:
    # 'SQL 답변:' 이후의 텍스트만 추출
    sql_start = response.find("SQL 답변:")
    if sql_start == -1:
        return "SQL 답변을 찾을 수 없습니다."
    # 'SQL 답변:' 이후의 텍스트 반환
    return response[sql_start + len("SQL 답변:"):].strip()


In [73]:
response = llm_chain.run(question=question)
print(response)

```sql
SELECT COUNT(*) AS donation_count
FROM ch_acq_TB
WHERE event_date = DATE_FORMAT(CURRENT_DATE(), '%Y-%m-02')
  AND source_medium = 'youtube/video';
```
